In [1]:
import sqlite3
from sqlite3 import Error
import pandas as pd
import numpy as np

# Loading CSV file

In [2]:
df_anime = pd.read_csv(r'data\myAnimelist-No_Hentai 2.csv').drop(columns='genre')
df_anime.rename(columns={"animeID":'anime_id'},inplace=True)
df_rating = pd.read_csv(r'data\rating.csv')

In [3]:
print(df_rating.shape)
# filtering for computational reasons & file saving size
df_rating_filtered = df_rating[df_rating.groupby("user_id")['user_id'].transform('size') > 500]
df_rating_filtered

(7813737, 3)


,user_id,anime_id,rating
1614,17,6,7
1615,17,19,10
1616,17,30,9
1617,17,32,10
1618,17,43,8
...,...,...,...
7812698,73502,18893,9
7812699,73502,19151,5
7812700,73502,19221,6
7812701,73502,19367,6


In [4]:
display(df_anime)
display(df_rating)

,anime_id,name,type,source,episodes,duration,rating,score,scored_by,rank,popularity,members,favorites
0,1,Cowboy Bebop,TV,Original,26,0:24:00,R - 17+ (violence & profanity),8.81,405664,26,39,795733,43460
1,5,Cowboy Bebop: Tengoku no Tobira,Movie,Original,1,1:55:00,R - 17+ (violence & profanity),8.41,120243,164,449,197791,776
2,6,Trigun,TV,Manga,26,0:24:00,PG-13 - Teens 13 or older,8.30,212537,255,146,408548,10432
3,7,Witch Hunter Robin,TV,Original,26,0:25:00,PG-13 - Teens 13 or older,7.33,32837,2371,1171,79397,537
4,8,Bouken Ou Beet,TV,Manga,52,0:23:00,PG - Children,7.03,4894,3544,3704,11708,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5555,38923,Sinbi Apateu: Ghost Ball Bimil,TV,Original,24,0:24:00,G - All Ages,7.00,4,13056,15281,30,1
5556,38961,Takoyaki Story,Music,Original,1,0:01:00,R+ - Mild Nudity,3.28,25,13254,15083,46,0
5557,38985,Birthday Wonderland,Movie,Book,1,1:55:00,G - All Ages,6.61,327,5504,9195,835,2
5558,39022,Ling Feng Zhe,ONA,Original,7,0:24:00,PG-13 - Teens 13 or older,6.08,38,11923,10455,467,2


,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1
...,...,...,...
7813732,73515,16512,7
7813733,73515,17187,9
7813734,73515,22145,10
7813735,73516,790,9


In [5]:
# Database Path object for easier calling
db_path = r"db\anime.db"

# Create a new database

In [6]:
# Create a new database file

def create_database(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    # connect to database file
    try:
        conn = sqlite3.connect(db_file)
        print(f'sqlite3 version: {sqlite3.version}')
        print('Created Database')
    # print error if connection can not be established
    except Error as error:
        print(error)
        print('Could not create Database')
    # close connection
    finally:
        if conn:
            conn.close()
    

# run the above function (save database file to path)
if __name__ == '__main__':
    create_database(db_path)

sqlite3 version: 2.6.0
Created Database


In [7]:
# # Create a new database that resides in the memory (RAM)

# def create_database():
#     """ create a database connection to a database that resides
#         in the memory
#     """
#     conn = None;
#     # connect to memory
#     try:
#         conn = sqlite3.connect(':memory:')
#         print(f'sqlite3 version: {sqlite3.version}')
#         print('Created Database')
#     # print error if connection can not be established
#     except Error as error:
#         print(error)
#         print('Could not create Database')
#     # close connection
#     finally:
#         if conn:
#             conn.close()
    

# # run the above function
# if __name__ == '__main__':
#     create_database()


# Create Tables

In [8]:
# function for establish connection to SQLite3 for editing database

def establish_connection(db_file):
    """ 
    creates a database connection to a SQLite database file
    
    :returns: Connection object or None
    """
    conn = None
    # connect to database file
    try:
        conn = sqlite3.connect(db_file)
        print(f'sqlite3 version: {sqlite3.version}')
        print('Connected to Database path')
    # print error if connection can not be established
    except Error as error:
        print(error)
        print('Could not create Database')
    
    return conn

In [9]:
# function for creating table in database

def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    print('Creating tables')
    try:
        # make object cursor from connect
        c = conn.cursor()
        # from cursor use execute to create table
        c.execute(create_table_sql)
    except Error as error:
        print(error)

In [10]:
# Create database schema

def main():
    
    sql_create_anime_table = """ CREATE TABLE IF NOT EXISTS Anime (
                                        anime_id integer PRIMARY KEY NOT NULL,
                                        name text NOT NULL,
                                        type text,
                                        source text,
                                        episodes integer,
                                        duration datetime,
                                        rating text,
                                        score real,
                                        scored_by integer,
                                        rank integer,
                                        popularity integer,
                                        members integer,
                                        favorites integer,
                                        FOREIGN KEY (anime_id) REFERENCES rating (anime_id)
                                    ); """

    sql_create_rating_table = """CREATE TABLE IF NOT EXISTS Rating (
                                    anime_id integer PRIMARY KEY NOT NULL,
                                    user_id integer NOT NULL,
                                    rating integer,
                                    FOREIGN KEY (anime_id) REFERENCES anime (anime_id)
                                    
                                );"""

    # create a database connection
    conn = establish_connection(db_path)

    # create tables
    if conn is not None:
        # create Anime table
        create_table(conn, sql_create_anime_table)

        # create Rating table
        create_table(conn, sql_create_rating_table)
        
        print("Created tables")
    else:
        print("Error! cannot create the database connection.")
        
    

    
if __name__ == '__main__':
    main()

sqlite3 version: 2.6.0
Connected to Database path}
Creating tables
Creating tables
Created tables


In [11]:
display(df_anime.head(5))
display(df_rating.head(5))

,anime_id,name,type,source,episodes,duration,rating,score,scored_by,rank,popularity,members,favorites
0,1,Cowboy Bebop,TV,Original,26,0:24:00,R - 17+ (violence & profanity),8.81,405664,26,39,795733,43460
1,5,Cowboy Bebop: Tengoku no Tobira,Movie,Original,1,1:55:00,R - 17+ (violence & profanity),8.41,120243,164,449,197791,776
2,6,Trigun,TV,Manga,26,0:24:00,PG-13 - Teens 13 or older,8.30,212537,255,146,408548,10432
3,7,Witch Hunter Robin,TV,Original,26,0:25:00,PG-13 - Teens 13 or older,7.33,32837,2371,1171,79397,537
4,8,Bouken Ou Beet,TV,Manga,52,0:23:00,PG - Children,7.03,4894,3544,3704,11708,14


,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


# Populating Tables from csv files

In [12]:
# exporting data from dataframe 'anime' to table anime
df_anime.to_sql('Anime',sqlite3.connect(db_path), if_exists='replace')

# export data from dataframe 'rating' to table rating 
df_rating.to_sql('Rating',sqlite3.connect(db_path), if_exists='replace')

In [13]:
# close connection to database

def close_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        conn.close()
        print('Connection successfully closed')
    except Error as error:
        print(error)
        print('Could not close connection')
# run the above function (save database file to path)
if __name__ == '__main__':
    close_connection(db_path)

Connection successfully closed


# Insert data into Tables

In [14]:
column_names

NameError: name 'column_names' is not defined

In [15]:
df_anime.anime_id.max

<bound method NDFrame._add_numeric_operations.<locals>.max of 0           1
1           5
2           6
3           7
4           8
        ...  
5555    38923
5556    38961
5557    38985
5558    39022
5559    39116
Name: anime_id, Length: 5560, dtype: int64>

In [16]:
column_names = list(df_anime.columns)
print(column_names)

vinland_saga = (39200,"Vinland Saga",5,6,24,'0:24:00',4,8.71,363439,41,134,737881,20182)
demon_slayer = (39201,"Demon Slayer",5,6,26,'0:23:00',4,8.61,966203,63,22,1567326,72436)
dororo = (39202,"Dororo",5,6,24,'0:24:00',4,8.19,341122,322,137,730890,12071)
Attack_on_Titan_season_3 = (39203,'Attack on Titan Season 3',5,6,12,'0:23:00',4,8.62,877466,64,32,1370047,17234)
my_hero_season_3 = (39204,"My Hero Academia Season 3",5,6,25,'0:23:00',3,8.21,916942,294,30,1452482,14255)
jujutsu_kaisen = (39205,"Jujutsu Kaisen",5,6,24,'0:23:00',4,8.79,441907,26,85,955531,39622)
mob_psycho_100_season_2 = (39206,"Mob Psycho 100 Season 2",5,14,13,'0:23:00',3,8.83,542236,22,99,903843,22297)

info_to_insert = [vinland_saga,demon_slayer,dororo,Attack_on_Titan_season_3,my_hero_season_3,jujutsu_kaisen,mob_psycho_100_season_2]

# for col,data in column_names,:
#     global() [new_anime_data] = [col,data]

['anime_id', 'name', 'type', 'source', 'episodes', 'duration', 'rating', 'score', 'scored_by', 'rank', 'popularity', 'members', 'favorites']


In [17]:
vinland_saga

(39200,
 'Vinland Saga',
 5,
 6,
 24,
 '0:24:00',
 4,
 8.71,
 363439,
 41,
 134,
 737881,
 20182)

In [26]:
def add_new_anime(conn,data_to_insert):
#     conn = establish_connection(db_path)
    
    
    sql = f""" INSERT into Anime(anime_id, name, type, source, episodes, duration, rating, score, scored_by, rank, popularity, members, favorites) 
                                            VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?);"""
    cur = conn.cursor()
    cur.execute(sql, data_to_insert)
    conn.commit()
    return cur.lastrowid
    

In [27]:
# function for creating table in database


    
def main():
    # create a database connection
    conn = establish_connection(db_path)

    # insert new anime data
    with conn:
        for x in info_to_insert:
            add_new_anime(conn, x)


        print("Successfully inserted new data into table")

    
    
    
if __name__ == '__main__':
    main()

sqlite3 version: 2.6.0
Connected to Database path}
Successfully inserted new data into table


# Querying data with conditions and save to Pandas Dataframe

In [ ]:
conn = sqlite3.connect(db_path)

In [ ]:

df_score_greater_then_8 = pd.read_sql_query("""SELECT * 
                                               from Anime 
                                               where score>8 
                                               order by score desc;""", conn)
df_score_greater_then_8

In [ ]:
df_rank_less_then_100 = pd.read_sql_query("Select * from Anime where rank<100 order by rank asc;", conn)


df_rank_less_then_100


In [ ]:
fullmetal_alchemist_bh_user_ratings = pd.read_sql_query(""" select * from Rating r
                                                            inner join Anime a on a.anime_id = r.anime_id
                                                            where a.anime_id = 5114
                                                            group by user_id
                                                            order by a.anime_id asc

                                                            ;""",conn)
                  
                  
fullmetal_alchemist_bh_user_ratings                      

In [ ]:
most_frequent_raters = pd.read_sql_query(""" select count(r.user_id) total_occurrence,r.user_id, avg(r.rating) avg_rating from Rating r
                                             inner join Anime a on a.anime_id = r.anime_id
                                             group by r.user_id
                                             order by total_occurrence desc

                                             limit 10

                                            ;""",conn)
                  
                            
                  
most_frequent_raters                  
                      

In [ ]:
least_frequent_raters = pd.read_sql_query("""select count(r.user_id) total_occurrence,r.user_id, avg(r.rating) avg_rating from Rating r
                                             inner join Anime a on a.anime_id = r.anime_id
                                             group by r.user_id
                                             order by total_occurrence asc

                                             limit 10

                                            ;""",conn)
                  
                            
                  
least_frequent_raters                  
                      

In [ ]:
total_users_rating_per_anime = pd.read_sql_query(""" select count(r.user_id) total_occurrence, a.* 
                                                     from Anime a
                                                     inner join Rating r on a.anime_id = r.anime_id
                                                     group by a.anime_id
                                                     order by total_occurrence desc



                                                    ;""",conn)

                            
                  
                  
total_users_rating_per_anime                      

In [ ]:
scores_classified = pd.read_sql_query("""select *,
                                         case
                                             when score >= 7.5 then 'highly_scored_game'
                                             when 6.0 >= score <= 7.49 then 'moderately_scored_game'
                                             else 'poorly_scored_game'
                                         end as 'anime_scored_classification'    
                                         from Anime
                                     ;""",conn)
scores_classified